# **Get Tweets in Real Time using Twitter API**

We are using Tweepy library to route all the Live tweets to socket server.
<br> This server is hosted in Cloud.

Import tweepy library

In [1]:
import tweepy

Import OAuthHandler.
<br>It is used for Authentication protocol (OAuth) of Twitter API.

In [2]:
# from tweepy import OAuthHandler

Import Stream & StreamListner
<br>This actually listen to streaming data getting from the socket

In [3]:
# from tweepy import Stream 
# from tweepy.streamin import StreamListener
from tweepy.asynchronous import AsyncStream

Import socket. It is used to create socket connection with Localhost i.e. local sercer.
<br>Json is used because tweeter data comes in json format.

In [4]:
import socket
import json

To connect to Twitter API, we need these 4 Keys.

In [5]:
import os
from dotenv import load_dotenv
load_dotenv() # take environment variables from .env

True

In [6]:
consumer_key = os.getenv("consumer_key")
consumer_secret = os.getenv("consumer_secret")
access_token = os.getenv("access_token")
access_token_secret = os.getenv("access_token_secret")

### Class TweetsListner
1. It is listening to the stream via StreamListener.
2. constructor function (init) : initializing the socket.
3. data contains the tweets json coming from the stream.
4. In this json object field 'text' contains the actual Tweet.
5. The actual tweet is extracted & sent to the client socket.
6. Error handling is done to catch & throw errors.

In [7]:
class TweetsListener(AsyncStream):
    def __init__(self, consumer_key, consumer_secret, access_token, access_token_secret, csocket):
        self.client_socket = csocket
        super().__init__(consumer_key, consumer_secret, access_token, access_token_secret)
        
    async def on_data(self, raw_data):
        try:
            msg = json.loads(raw_data)
            
            # print(msg['text'].encode('utf-8'))
            # print(msg['created_at'].encode('utf-8'))
            output = json.dumps(msg['text'], ensure_ascii=False)
            print(output)
            
            self.client_socket.send(msg['text'].encode('utf-8'))
            return True
        except Exception as e:
            print("Error on_data: %s" % str(e))
        return True
    
    async def on_status(self, status):
        print(status.text)
        
    # def on_error(self, status):
    #     print(status)
    #     return True

### Functio SendData():
1. auth is doing Authentication check using OAuthHandler. It uses 4 keys of Twitter APIs.
2. consumer_key & consumer_secret are like username & access_access_token & access_secret are like password.
3. twitter_stream will get the actual twitter live stream data.
4. It'll call Stream with auth & call the Class TweetsListner & pass c_socket info to the class.
5. Fromt he stream of tweets, filter & take only tweets which contains "Trunp" word.

In [8]:
def sendData(c_socket): # c_socket or client socket
    twitter_stream = TweetsListener(consumer_key, consumer_secret, access_token, access_token_secret, c_socket)
    twitter_stream.filter(track=['BJP'])
#     auth = OAuthHandler(consumer_key, consumer_secret)
#     auth.set_access_token(access_token, access_token_secret)
    
#     twitter_stream = Stream(auth, TweetsListener(c_socket)) 
#     twitter_stream.filter(track=['BTS'])    

### Socket Creation:
1. Socket is created by using socket function
2. Host is localhost, which is our case is IP of where this Jupyter notebook is running.
3. Port is 5555 (It can be anything, unique for this service)
4. Now bind host & port address to the socket.


In [9]:
#if __name__ == "__main__":
s = socket.socket()     # Create a socket object
host = '172.16.117.15'  # Get local machine name
port = 6677             # Reserce a port for your service
s.bind((host, port))    # Bind to the port
print("Listening on port: %s" % str(port))

Listening on port: 6677


socket will wait & listen for few seconds.
<br>After that connection is made.

In [10]:
s.listen(5)            # Now wait for client connection.
c, addr = s.accept()   # Establish connection with client.
print("Recived request from: " + str(addr))

Recived request from: ('172.16.117.15', 49166)


## Send live Tweets in real time to the Socket
Function sendData will send all Tweets containing word 'Trump' to socket 'c' in real time.
<br>This will be consumed in the next code of Spark Streaming to do Trend Analysis.

In [11]:
sendData(c)

In [12]:
# Call close method on the socket objects to release their resources
# c.close()

"RT @kapsology: 4 Delhi schools are in the top 10 government schools in India\n\nZERO from any BJP ruled state https://t.co/p4fHIUwgvC"
"RT @sangram_enm: Perfect prey for BJP"
"RT @_narsinghshukla: “SELL INDIA CAMPAIGN” of BJP Govt!\n\nमोदी सरकार का “भारत बेचो अभियान” !\n\nNOW ON SALE -:\n\n• Bharat Earth Movers Ltd.\n• S…"
"RT @rssurjewala: “SELL INDIA CAMPAIGN” of BJP Govt!\n\nमोदी सरकार का “भारत बेचो अभियान” !\n\nNOW ON SALE -:\n\n• Bharat Earth Movers Ltd.\n• Shipp…"
"RT @NewsArenaIndia: Telangana News\n\nTRS CM, MLAs &amp; MPs will protest against PM Narendra Modi today at Dharna Chowk.\n\nThe more they protest…"
"RT @DhoteAnup09: अनिल देशमुख की गिरफ्तारी पर शरद पवार का फूटा गुस्सा, BJP से बोले- हर दिन, हर घंटे की कीमत वसूलेंगे https://t.co/RGIk2UT0Tp…"
"RT @AleemIlahi: BJP trying to steal mandate illegally, responsibility on Secular parties to join hands and stop it in Gulbarga Mahanagara P…"
"@kpmaurya1 Bjp hatao desh bachao"
"Aap bjp ke liye bhut achha kaam kare rehe aap bhut a